# Quantum Error Correction

### Three bit effective qubits
The quantum analog to the classical three bit replication code 
$$
\begin{align}
a \vert 0 \rangle + b\vert 1\rangle \mapsto~
&~~CX_{13}CX_{12}\big( a \vert 0 \rangle + b\vert 1\rangle \big)\vert 00\rangle \\
&= CX_{13}\big( a \vert 00 \rangle + b\vert 11\rangle \big)\vert 0\rangle \\
&= a \vert 000\rangle + b\vert 111\rangle  
\end{align}
$$
yields the effective qubit state $a \vert 000\rangle + b\vert 111\rangle$. 


## Bit errors
If transmission of the three bit effective qubit state is subject to bit wise binary symmetric channel noise with via the Kraus operator $E_{p,\text{bit}}$ such that an $n$-qubit state's density operators $\rho$ are modified as
$$
E_{p,\text{bit}}(\rho) = (1-p)\rho + p X\rho X 
$$
then error correction can be done with 
$$
{\cal E}_{\text{corr}} = P_0 + \sum_i X_i P_i
$$
where $X_i$ is Pauli X applied to the $i$th qubit and
$$
\begin{align}
P_0 = \vert 000 \rangle \langle  000 \vert + \vert 111 \rangle \langle 111\vert ,\\
P_1 = \vert 100 \rangle \langle  100 \vert + \vert 011 \rangle \langle 011\vert ,\\
P_0 = \vert 010 \rangle \langle  010 \vert + \vert 101 \rangle \langle 101\vert, \\
P_0 = \vert 001 \rangle \langle  001 \vert + \vert 110 \rangle \langle 110\vert \,.\\
\end{align}
$$


The probability of a 2 bit error is the same as in classical 3 bit repetition code. Two qubit errors are not corrected. 

The matrix of ${\cal E}_{\text{corr}}$ is calculated below. 

In [3]:
import numpy as np
import pennylane as qml

first_kets = np.array([[0,0,0],[1,0,0],[0,1,0],[0,0,1]])
second_kets = np.array([(ket + 1)%2 for ket in first_kets])

Ps = np.array([np.zeros((8,8)) for _ in range(4)])
for i in range(4):
    first_index  = np.ravel_multi_index(first_kets[i], [2]*3)
    second_index = np.ravel_multi_index(second_kets[i],[2]*3)
    Ps[i,first_index,first_index] = 1
    Ps[i,second_index,second_index] = 1

X = np.array([[0,1],[1,0]])
I = np.eye(2)
Xs = [np.eye(8) ,
      np.kron(np.kron(X,I),I),
      np.kron(np.kron(I,X),I),
      np.kron(np.kron(I,I),X)
     ]

E = sum([ Xs[i]@Ps[i] for i in range(4)])
E = qml.QubitUnitary(E, wires =range(3))
E # The fluent reader can see here that E is the right matrix. 

QubitUnitary(array([[1., 1., 1., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 1., 1., 1.]]), wires=[0, 1, 2])

In [631]:
corrupted_logical_qubit = [0,0,1]
corrupted_state = np.zeros(8) # Initializing
corrupted_state[np.ravel_multi_index(corrupted_logical_qubit,[2]*3)] = 1 # one non-zero component 
E = sum( [ Xs[i]@Ps[i] for i in range(4)])
E@corrupted_state
#Should be corrected to 000 = [1,0,....,0] 

array([1., 0., 0., 0., 0., 0., 0., 0.])

This matrix is equivalent to $MCX_{120}CX_{02}CX_{01}$. but getting PennyLane to show that matrix to me seems a little difficult. 

In [681]:
# Not working for me :(
mat1 = (qml.CNOT(wires = [0,1])@qml.I(wires=[2])).matrix()
mat2 = (qml.CNOT(wires = [0,2])@qml.I(wires=[1])).matrix()
mat3 = (qml.Toffoli(wires = [1,2,0])).matrix()
mat3@mat2@mat1

array([[1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 0., 0., 1., 0.]])

## Phase flip

For correcting a phase flip (which is a flip by $Z$, analagous to how a bit flip is a flip by $X$)
further encode with $H^{\otimes 3}$ to get 
$$
\begin{align}
a \vert 0 \rangle + b\vert 1\rangle \mapsto~
&~~H^{\otimes 3} CX_{13}CX_{12}\big( a \vert 0 \rangle + b\vert 1\rangle \big)\vert 00\rangle \\
&= H^3 \big(a\vert 00 0\rangle + b\vert 111\rangle\big) \\ 
&=  a\vert +++ \rangle + b\vert ---\rangle\big) \\ 
\end{align}
$$

In transmission, a phase flip may occur, e.g in the first component giving 
$$
\vert -++ \rangle + b\vert +--\rangle\big) \\ 
$$
This can be corrected by applying $H^3$ to get 
$$
\vert 100 \rangle + b\vert 011\rangle\big) \\ 
$$
and then applying the bit error correction. 

I'm a little concerned that bit flip and phase flip corrections seem to requre diferent pre-transmission encoding; surely there is a way to do both. 